# Introduction.

## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto.

### Contains the three exercises.

### Table of Contents:
1. <a href="#item1">Exercise 1</a>

2. <a href="#item2">Exercise 2</a>

3. <a href="#item3">Exercise 3</a>    

#### How to list:
##### How to convert addresses into their equivalent latitude and longitude values. 
##### How to use the Foursquare API to explore neighborhoods in Toronto. 
##### How to explore the most common venue categories in each neighborhood. 
##### How to group the neighborhoods into clusters using the k-means clustering algorithm. 
##### How to use the Folium library to visualize the neighborhoods in Toronto and their emerging clusters.

<a id='item1'></a>
# 1. First exercise.

## 1. Importing libraries

In [9]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests 
from bs4 import BeautifulSoup 
import wget
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


## 2. Download Dataset

In [10]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(URL) 

## 3. Scraping...

In [11]:
scraped_data = BeautifulSoup(res.text, 'html.parser')

## 4. Cleaning...

In [12]:
data_rows = scraped_data.select('.wikitable tr')

In [13]:
data_header = data_rows[0]
data_content = data_rows[1:] 

In [14]:
data_columns = []

for column in data_header.select('th'): 
    column_name = column.getText()
    data_columns.append(column_name.replace('\n', ''))  
    
data_columns

['Postal code', 'Borough', 'Neighborhood']

In [15]:
final_data = []

for row in data_content:
    data_row = []
    for data in row.select('td'): 
        data_row.append(data.getText().replace('\n', ''))
    final_data.append(data_row)

## 5. Creating Pandas DataFrame

In [16]:
torontoDF = pd.DataFrame(data=final_data, columns=data_columns)
torontoDF.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [17]:
torontoDF.tail()

,Postal code,Borough,Neighborhood
175,M5Z,Not assigned,
176,M6Z,Not assigned,
177,M7Z,Not assigned,
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...
179,M9Z,Not assigned,


## 6. Cleaning Pandas DataFrame
##### 1. Ignore cells with a borough that is Not assigned.
##### 2. More than one neighborhood can exist in one postal code area. These rows will be combined into one row with the neighborhoods separated with a comma.
##### 3. Reset relative number of rows.

In [18]:
torontoDF = torontoDF[torontoDF.Borough != 'Not assigned']
torontoDF['Neighborhood'] = torontoDF['Neighborhood'].apply(lambda n: n.replace(' / ', ', '))
torontoDF = torontoDF.reset_index(drop=True)
torontoDF.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## 7. Saving result to csv file

In [19]:
torontoDF.to_csv('Toronto_pc.csv')

## 8. Number of rows

In [20]:
torontoDF.shape

(103, 3)

<a id='item2'></a>
# 2. Second exercise.

## 9. We need to get the latitude and the longitude coordinates of each neighborhood.

In [21]:
Geospatial_data = wget.download('https://cocl.us/Geospatial_data')

100% [................................................................................] 2891 / 2891

In [22]:
GeospatialDF = pd.read_csv(r'Geospatial_Coordinates.csv')
GeospatialDF.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## 10. Combining both data frames

In [23]:
Final_data = pd.merge(torontoDF, GeospatialDF, left_on='Postal code', right_on='Postal Code', how='inner')
Final_data.head()

,Postal code,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M7A,43.662301,-79.389494


## 11. Cleaning duplicated Postal Code column

In [24]:
Final_data = Final_data.drop(['Postal Code'], axis=1)
Final_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [25]:
Final_data.shape

(103, 5)

<a id='item3'></a>
# 3. Third exercise

## 12. Map of Toronto with neighborhoods superimposed on top.

In [26]:
# Because of Service timed out from geocode
latitude = 43.651070
longitude = -79.347015

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Final_data['Latitude'], Final_data['Longitude'], Final_data['Borough'], Final_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## 13. Let's slice the original dataframe and create a new dataframe of the North York data.

In [32]:
northyork_data = Final_data[Final_data['Borough'] == 'North York'].reset_index(drop=True)
northyork_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


## 14. Map of Toronto with North York borough superimposed on top

In [33]:
# Because of Service timed out from geocode
latitude = 43.7668114
longitude = -79.4162979

# create map of New York using latitude and longitude values
map_toronto_northyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(northyork_data['Latitude'], northyork_data['Longitude'], northyork_data['Borough'], northyork_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_northyork)  
    
map_toronto_northyork

 ## 15. Define Foursquare Credentials

In [34]:
CLIENT_ID = 'PKOBLYCFF1NEKHLDPTGWIJKRMGJENMPWPL4WEEZAKZZKL2L1' # your Foursquare ID
CLIENT_SECRET = 'M4LTSJGUGTGBPGL1RWX20F3TD3NPTSQRAZE3X5WCKDGEMZEG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PKOBLYCFF1NEKHLDPTGWIJKRMGJENMPWPL4WEEZAKZZKL2L1
CLIENT_SECRET:M4LTSJGUGTGBPGL1RWX20F3TD3NPTSQRAZE3X5WCKDGEMZEG


## 16. Let's explore the first neighborhood in our dataframe.

In [35]:
northyork_data.loc[0, 'Neighborhood']

'Parkwoods'

## 17. Get the neighborhood's latitude and longitude values.

In [36]:
neighborhood_latitude = northyork_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = northyork_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = northyork_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


## 18. Let's get the top 100 venues that are in North York within a radius of 500 meters.

In [38]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e871879211536001b8a7eae'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

## 19. Let's borrow the get_category_type function from the Foursquare lab.

In [39]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## 20. Cleaning the json and structure it into a pandas dataframe

In [40]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\jgonzalez\Miniconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,GTA Restoration,Fireworks Store,43.753396,-79.333477
2,Variety Store,Food & Drink Shop,43.751974,-79.333114


## 21. Number of venues returned by Foursquare

In [41]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


## 22. Let's create a function to repeat the same process to all the neighborhoods in North York

In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## 23. Creating a new dataframe with North York venues

In [43]:
NorthYork_venues = getNearbyVenues(names=northyork_data['Neighborhood'],
                                   latitudes=northyork_data['Latitude'],
                                   longitudes=northyork_data['Longitude']
                                  )


Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale
Downsview
York Mills West
Willowdale


## 24. Let's explore the resulting dataframe

In [44]:
print(NorthYork_venues.shape)
NorthYork_venues.head()

(243, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,GTA Restoration,43.753396,-79.333477,Fireworks Store
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [46]:
NorthYork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Don Mills,24,24,24,24,24,24
Downsview,16,16,16,16,16,16
"Fairview, Henry Farm, Oriole",70,70,70,70,70,70
Glencairn,4,4,4,4,4,4
Hillcrest Village,5,5,5,5,5,5
Humber Summit,1,1,1,1,1,1


In [47]:
print('There are {} uniques categories.'.format(len(NorthYork_venues['Venue Category'].unique())))

There are 104 uniques categories.


## 25. Analyze Each Neighborhood

In [48]:
# one hot encoding
NorthYork_onehot = pd.get_dummies(NorthYork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
NorthYork_onehot['Neighborhood'] = NorthYork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [NorthYork_onehot.columns[-1]] + list(NorthYork_onehot.columns[:-1])
NorthYork_onehot = NorthYork_onehot[fixed_columns]

NorthYork_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Fireworks Store,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [49]:
NorthYork_onehot.shape

(243, 105)

## 26. Let's explore the mean of the frequency of occurrence of each category by neighborhood

In [50]:
NorthYork_grouped = NorthYork_onehot.groupby('Neighborhood').mean().reset_index()
NorthYork_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Fireworks Store,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.0000,0.000000,0.000,0.000000,0.0000,0.000000,0.105263,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.052632,0.000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.105263,0.00,0.000000,0.00,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.052632,0.052632,0.000000,0.052632,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.0,0.0000,0.0000,0.052632,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.0,0.052632,0.000000,0.000000,0.000000,0.000,0.052632,0.0,0.052632,0.000,0.0,0.0,0.00,0.000,0.052632,0.000000,0.052632,0.000000,0.052632,0.00,0.000000,0.000,0.052632,0.000000,0.052632,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.0000,0.000000,0.000,0.000000,0.0000,0.000000,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000,0.250000,0.000000,0.250000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.0,0.0000,0.0000,0.000000,0.00,0.000,0.0,0.000000,0.250000,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000,0.0,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.0000,0.040000,0.000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.040,0.040000,0.000000,0.000000,0.000000,0.000000,0.080000,0.04,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.040000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.040000,0.0400,0.000000,0.000000,0.0,0.0000,0.0000,0.000000,0.04,0.000,0.0,0.080000,0.000000,0.000000,0.040000,0.040000,0.000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.040000,0.0,0.080000,0.000,0.0,0.0,0.04,0.000,0.080000,0.000000,0.080000,0.000000,0.000000,0.04,0.000000,0.000,0.000000,0.000000,0.04000

In [51]:
NorthYork_grouped.shape

(18, 105)

## 27. Showing the top 5 most common venues for each neighborhood

In [52]:
num_top_venues = 5

for hood in NorthYork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = NorthYork_grouped[NorthYork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
                 venue  freq
0          Coffee Shop  0.11
1                 Bank  0.11
2        Shopping Mall  0.05
3  Fried Chicken Joint  0.05
4        Deli / Bodega  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2                 Café  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.08
1          Restaurant  0.08
2         Coffee Shop  0.08
3  Italian Restaurant  0.08
4         Pizza Place  0.08


----Don Mills----
                 venue  freq
0          Coffee Shop  0.08
1           Restaurant  0.08
2                  Gym  0.08
3  Japanese Restaurant  0.08
4           Beer Store  0.08


----Downsview----
            venue  freq
0   Grocery Store  0.19
1            Park  0.12
2    Liquor Store  0.06
3  Discount Store  0.06
4    Home Service  0.06


## 28. A function to sort the venues in descending order.

In [53]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## 29. Let's put data into a pandas dataframe with the top 10 venues for each neighborhood.

In [54]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
NorthYork_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
NorthYork_neighborhoods_venues_sorted['Neighborhood'] = NorthYork_grouped['Neighborhood']

for ind in np.arange(NorthYork_grouped.shape[0]):
    NorthYork_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NorthYork_grouped.iloc[ind, :], num_top_venues)

NorthYork_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Restaurant,Bridal Shop,Diner,Pharmacy,Sandwich Place,Deli / Bodega,Shopping Mall,Pizza Place
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Distribution Center,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
2,"Bedford Park, Lawrence Manor East",Restaurant,Sandwich Place,Italian Restaurant,Pizza Place,Coffee Shop,Thai Restaurant,Liquor Store,Juice Bar,Comfort Food Restaurant,Fast Food Restaurant
3,Don Mills,Restaurant,Coffee Shop,Beer Store,Japanese Restaurant,Gym,Asian Restaurant,Clothing Store,Chinese Restaurant,Supermarket,Dim Sum Restaurant
4,Downsview,Grocery Store,Park,Baseball Field,Gym / Fitness Center,Home Service,Hotel,Discount Store,Liquor Store,Shopping Mall,Food Truck


## 30. Run k-means to cluster the neighborhood into 5 clusters.

In [55]:
# set number of clusters
kclusters = 5

NorthYork_grouped_clustering = NorthYork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NorthYork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 4, 0, 4, 0, 4, 2, 3])

## 31. A new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [85]:
# add clustering labels
# NorthYork_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

NorthYork_merged = northyork_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
NorthYork_merged = NorthYork_merged.join(NorthYork_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# drop postal code column. I don't need it.
NorthYork_merged = NorthYork_merged.drop(['Postal code'], axis=1)

# One row has NaN values. Removed for being problematic
NorthYork_merged = NorthYork_merged.dropna()

# Resulting Clusters labels are float. Converted to int because folium.CircleMarker need int values to color.
NorthYork_merged['Cluster Labels'] = NorthYork_merged['Cluster Labels'].astype(int)

NorthYork_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,43.753259,-79.329656,0,Park,Food & Drink Shop,Fireworks Store,Discount Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
1,North York,Victoria Village,43.725882,-79.315572,4,Intersection,Pizza Place,Hockey Arena,Portuguese Restaurant,Coffee Shop,Discount Store,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
2,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,4,Clothing Store,Accessories Store,Boutique,Furniture / Home Store,Event Space,Miscellaneous Shop,Coffee Shop,Vietnamese Restaurant,Electronics Store,Cosmetics Shop
3,North York,Don Mills,43.745906,-79.352188,4,Restaurant,Coffee Shop,Beer Store,Japanese Restaurant,Gym,Asian Restaurant,Clothing Store,Chinese Restaurant,Supermarket,Dim Sum Restaurant
4,North York,Glencairn,43.709577,-79.445073,0,Park,Pizza Place,Pub,Japanese Restaurant,Clothing Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop


## 32. Visualizing the resulting clusters in a map

In [92]:
# create map
NorthYork_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map, 
markers_colors = []
for lat, lon, poi, cluster in zip(NorthYork_merged['Latitude'], NorthYork_merged['Longitude'], NorthYork_merged['Neighborhood'], NorthYork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],  
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(NorthYork_map_clusters)
       
NorthYork_map_clusters

## 33. Examine Cluster 1

In [86]:
NorthYork_merged.loc[NorthYork_merged['Cluster Labels'] == 0, NorthYork_merged.columns[[1] + list(range(5, NorthYork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,Park,Food & Drink Shop,Fireworks Store,Discount Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
4,Glencairn,Park,Pizza Place,Pub,Japanese Restaurant,Clothing Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
11,Downsview,Grocery Store,Park,Baseball Field,Gym / Fitness Center,Home Service,Hotel,Discount Store,Liquor Store,Shopping Mall,Food Truck
13,Downsview,Grocery Store,Park,Baseball Field,Gym / Fitness Center,Home Service,Hotel,Discount Store,Liquor Store,Shopping Mall,Food Truck
14,"North Park, Maple Leaf Park, Upwood Park",Construction & Landscaping,Bakery,Basketball Court,Park,Women's Store,Distribution Center,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega
17,Downsview,Grocery Store,Park,Baseball Field,Gym / Fitness Center,Home Service,Hotel,Discount Store,Liquor Store,Shopping Mall,Food Truck
21,Downsview,Grocery Store,Park,Baseball Field,Gym / Fitness Center,Home Service,Hotel,Discount Store,Liquor Store,Shopping Mall,Food Truck
22,York Mills West,Park,Convenience Store,Bank,Bar,Women's Store,Distribution Center,Concert Hall,Construction & Landscaping,Cosmetics Shop,Deli / Bodega


## 34. Examine Cluster 2

In [87]:
NorthYork_merged.loc[NorthYork_merged['Cluster Labels'] == 1, NorthYork_merged.columns[[1] + list(range(5, NorthYork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,"Willowdale, Newtonbrook",Piano Bar,Women's Store,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop


## 35. Examine Cluster 3

In [88]:
NorthYork_merged.loc[NorthYork_merged['Cluster Labels'] == 2, NorthYork_merged.columns[[1] + list(range(5, NorthYork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Humber Summit,Empanada Restaurant,Women's Store,Distribution Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop


## 36. Examine Cluster 4

In [89]:
NorthYork_merged.loc[NorthYork_merged['Cluster Labels'] == 3, NorthYork_merged.columns[[1] + list(range(5, NorthYork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,"Humberlea, Emery",Baseball Field,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant


## 37. Examine Cluster 5

In [90]:
NorthYork_merged.loc[NorthYork_merged['Cluster Labels'] == 4, NorthYork_merged.columns[[1] + list(range(5, NorthYork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Victoria Village,Intersection,Pizza Place,Hockey Arena,Portuguese Restaurant,Coffee Shop,Discount Store,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
2,"Lawrence Manor, Lawrence Heights",Clothing Store,Accessories Store,Boutique,Furniture / Home Store,Event Space,Miscellaneous Shop,Coffee Shop,Vietnamese Restaurant,Electronics Store,Cosmetics Shop
3,Don Mills,Restaurant,Coffee Shop,Beer Store,Japanese Restaurant,Gym,Asian Restaurant,Clothing Store,Chinese Restaurant,Supermarket,Dim Sum Restaurant
5,Don Mills,Restaurant,Coffee Shop,Beer Store,Japanese Restaurant,Gym,Asian Restaurant,Clothing Store,Chinese Restaurant,Supermarket,Dim Sum Restaurant
6,Hillcrest Village,Golf Course,Mediterranean Restaurant,Pool,Fast Food Restaurant,Dog Run,Women's Store,Diner,Concert Hall,Construction & Landscaping,Convenience Store
7,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Restaurant,Bridal Shop,Diner,Pharmacy,Sandwich Place,Deli / Bodega,Shopping Mall,Pizza Place
8,"Fairview, Henry Farm, Oriole",Clothing Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Restaurant,Women's Store,Tea Room,Juice Bar,Jewelry Store,Food Court
9,"Northwood Park, York University",Coffee Shop,Caribbean Restaurant,Massage Studio,Bar,Distribution Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
10,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Distribution Center,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
18,"Bedford Park, Lawrence Manor East",Restaurant,Sandwich Place,Italian Restaurant,Pizza Place,Coffee Shop,Thai Restaurant,Liquor Store,Juice Bar,Comfort Food Restaurant,Fast Food Restaurant
